# EDA Impacto Subtype Coordenadas

In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 40)
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [2]:
df = pd.read_csv('../../Data/02ParaLimpiar/02desastres_paralimpiar.csv', encoding='utf-8', delimiter=';', engine='python')

In [5]:
# Selección de columnas
time_subtype_impact_coords = ['Start Day', 'Start Month', 'Start Year', 'End Day', 'End Month', 'End Year', 'Disaster Type', 'Disaster Subtype', 'Total Deaths', 'Total Affected', 'Latitude', 'Longitude', 'Location']
# Create a subset dataframe with the selected columns
df_subset = df[time_subtype_impact_coords]

# Pivotear el dataframe para separar subtypes como columns
df_pivot = df_subset.pivot_table(index=['Start Day', 'Start Month', 'Start Year', 'End Day', 'End Month', 'End Year', 'Disaster Type', 'Total Deaths', 'Total Affected'], columns='Disaster Subtype', aggfunc='sum')

# Flatten the multi-level column index
df_pivot.columns = ['_'.join(col) for col in df_pivot.columns]

# Resetear index
df_pivot = df_pivot.reset_index()

# Definir lista de subtypes
subtypes = df['Disaster Subtype'].unique()

# Limpiar las columnas de coordenadas para cada subtype
for subtype in subtypes:
    lat_col = f'{subtype}_Latitude'
    lon_col = f'{subtype}_Longitude'
    loc_col = f'{subtype}_Location'
    
    if lat_col in df_pivot.columns and lon_col in df_pivot.columns:
        df_pivot[lat_col] = df_pivot[lat_col].apply(clean_coordinates)
        df_pivot[lon_col] = df_pivot[lon_col].apply(clean_coordinates)
    else:
        # Columnas de coords a NaN o Not applicable
        df_pivot[lat_col] = np.nan
        df_pivot[lon_col] = np.nan
    
    if loc_col in df_pivot.columns:
        df_pivot[loc_col] = df_pivot[loc_col].apply(clean_location)
    else:
        # Columna Location a NaN o Not applicable
        df_pivot[loc_col] = np.nan

# Resultado de la pivot table
print(df_pivot.head())

   Start Day  Start Month  Start Year  End Day  End Month  End Year  \
0        1.0          1.0        1964      1.0        1.0      1964   
1        1.0          1.0        1965      1.0        1.0      1965   
2        1.0          1.0        1967      1.0        1.0      1967   
3        1.0          1.0        1967      1.0        1.0      1967   
4        1.0          1.0        1968      1.0        1.0      1968   

  Disaster Type  Total Deaths  Total Affected Latitude_Ash fall  ...  \
0      Epidemic         598.0         10848.0               NaN  ...   
1    Earthquake           2.0         49350.0               NaN  ...   
2      Epidemic           2.0        200000.0               NaN  ...   
3      Epidemic        3029.0         13576.0               NaN  ...   
4      Epidemic          40.0            94.0               NaN  ...   

  Subsidence_Location Extra-tropical storm_Latitude  \
0                 NaN                           NaN   
1                 NaN         

In [8]:
# Iterar por todas las columnas subtype
for subtype in subtypes:
    subtype_col = f'{subtype}_Latitude'
    
    if subtype_col in df_pivot.columns:
        # Filtrar las rows con valores NaN
        filtered_df = df_pivot.dropna(subset=[subtype_col])
        
        # Revisar si filtered_df tiene rows
        if not filtered_df.empty:
            # Realizar análisis o visualización
            subtype_counts = filtered_df[subtype_col].value_counts()
            
            # Plot bar
            plt.figure()
            subtype_counts.plot(kind='bar')
            plt.title(f'{subtype} Distribución de Latitude')
            plt.xlabel('Latitude')
            plt.ylabel('Count')
            plt.show()
        else:
            print(f'No hay valores Latitude disponibles para subtype: {subtype}')

No data available for subtype: Drought
No data available for subtype: Ground movement
No data available for subtype: Ash fall
No data available for subtype: Rockfall
No data available for subtype: Tropical cyclone
No data available for subtype: nan
No data available for subtype: Bacterial disease
No data available for subtype: Landslide
No data available for subtype: Avalanche
No data available for subtype: Forest fire
No data available for subtype: Riverine flood
No data available for subtype: Convective storm
No data available for subtype: Viral disease
No data available for subtype: Mudslide
No data available for subtype: Tsunami
No data available for subtype: Heat wave
No data available for subtype: Land fire (Brush, Bush, Pasture)
No data available for subtype: Coastal flood
No data available for subtype: Cold wave
No data available for subtype: Flash flood
No data available for subtype: Parasitic disease
No data available for subtype: Locust
No data available for subtype: Worms
N

In [9]:
# Iterar por todas las columnas subtype
for subtype in subtypes:
    subtype_col = f'{subtype}_Longitude'
    
    if subtype_col in df_pivot.columns:
        # Filtrar las rows con valores NaN
        filtered_df = df_pivot.dropna(subset=[subtype_col])
        
        # Revisar si filtered_df tiene rows
        if not filtered_df.empty:
            # Realizar análisis o visualización
            subtype_counts = filtered_df[subtype_col].value_counts()
            
            # Plot bar
            plt.figure()
            subtype_counts.plot(kind='bar')
            plt.title(f'{subtype} Distribución de Longitude')
            plt.xlabel('Longitude')
            plt.ylabel('Count')
            plt.show()
        else:
            print(f'No hay valores Longitude disponibles para subtype: {subtype}')

No hay datos disponibles para subtype: Drought
No hay datos disponibles para subtype: Ground movement
No hay datos disponibles para subtype: Ash fall
No hay datos disponibles para subtype: Rockfall
No hay datos disponibles para subtype: Tropical cyclone
No hay datos disponibles para subtype: nan
No hay datos disponibles para subtype: Bacterial disease
No hay datos disponibles para subtype: Landslide
No hay datos disponibles para subtype: Avalanche
No hay datos disponibles para subtype: Forest fire
No hay datos disponibles para subtype: Riverine flood
No hay datos disponibles para subtype: Convective storm
No hay datos disponibles para subtype: Viral disease
No hay datos disponibles para subtype: Mudslide
No hay datos disponibles para subtype: Tsunami
No hay datos disponibles para subtype: Heat wave
No hay datos disponibles para subtype: Land fire (Brush, Bush, Pasture)
No hay datos disponibles para subtype: Coastal flood
No hay datos disponibles para subtype: Cold wave
No hay datos dis

In [10]:
# Iterar por todas las columnas subtype
for subtype in subtypes:
    subtype_col = f'{subtype}_Location'
    
    if subtype_col in df_pivot.columns:
        # Filtrar las rows con valores NaN
        filtered_df = df_pivot.dropna(subset=[subtype_col])
        
        # Revisar si filtered_df tiene rows
        if not filtered_df.empty:
            # Realizar análisis o visualización
            subtype_counts = filtered_df[subtype_col].value_counts()
            
            # Plot bar
            plt.figure()
            subtype_counts.plot(kind='bar')
            plt.title(f'{subtype} Distribución de Location')
            plt.xlabel('Location')
            plt.ylabel('Count')
            plt.show()
        else:
            print(f'No hay valores de Location disponibles para subtype: {subtype}')

No hay valores de Location disponibles para subtype: Drought
No hay valores de Location disponibles para subtype: Ground movement
No hay valores de Location disponibles para subtype: Ash fall
No hay valores de Location disponibles para subtype: Rockfall
No hay valores de Location disponibles para subtype: Tropical cyclone
No hay valores de Location disponibles para subtype: nan
No hay valores de Location disponibles para subtype: Bacterial disease
No hay valores de Location disponibles para subtype: Landslide
No hay valores de Location disponibles para subtype: Avalanche
No hay valores de Location disponibles para subtype: Forest fire
No hay valores de Location disponibles para subtype: Riverine flood
No hay valores de Location disponibles para subtype: Convective storm
No hay valores de Location disponibles para subtype: Viral disease
No hay valores de Location disponibles para subtype: Mudslide
No hay valores de Location disponibles para subtype: Tsunami
No hay valores de Location di